# Setting optimal MLP's hiperparameters for news dataset

In [ ]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 70)

In [2]:
# Read preprocessed news dataset
news_df = pd.read_csv('./prep_news_data.csv')
news_df

,rank( timedelta),rank( n_tokens_title),rank( n_tokens_content),rank( n_unique_tokens),rank( n_non_stop_words),rank( n_non_stop_unique_tokens),rank( num_hrefs),rank( num_self_hrefs),num_imgs,num_videos,rank( average_token_length),rank( num_keywords),data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,rank( kw_max_min),rank( kw_avg_min),kw_min_max,kw_max_max,rank( kw_avg_max),kw_min_avg,rank( kw_max_avg),rank( kw_avg_avg),rank( self_reference_min_shares),rank( self_reference_max_shares),rank( self_reference_avg_sharess),weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,rank( LDA_00),rank( LDA_01),rank( LDA_02),rank( LDA_03),rank( LDA_04),rank( global_subjectivity),rank( global_sentiment_polarity),rank( global_rate_positive_words),rank( global_rate_negative_words),rank( rate_positive_words),rank( rate_negative_words),rank( avg_positive_polarity),min_positive_polarity,max_positive_polarity,rank( avg_negative_polarity),rank( min_negative_polarity),max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,3.091313,0.746312,-0.830751,1.171875,-0.859076,1.250329,-0.773700,-0.247319,1.0,0.0,0.060832,-1.077791,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.809729,-0.897243,-0.895353,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.011874,1.107189,0.003762,0.133906,-0.019697,0.834551,-0.299737,0.407371,-0.186943,0.425334,-0.344559,0.257404,0.100000,0.70,-0.840613,-0.309042,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,3.091313,-0.627749,-0.629945,0.636396,-0.682170,1.037852,-1.082209,-0.770293,1.0,0.0,0.874264,-1.654145,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.506066,0.481113,0.164789,0.215657,0.071541,-1.206170,0.345029,0.255150,0.038674,0.167738,-0.092096,-0.900984,0.033333,0.70,1.303623,1.355812,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,3.091313,-0.627749,-0.877419,0.346405,-0.977397,-0.284000,-1.082209,-0.770293,1.0,0.0,-0.983430,-0.561454,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.314418,-0.753138,-0.722589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628018,-0.102896,-0.194219,-0.319837,1.102533,2.439542,2.017790,1.034661,-0.690612,1.106431,-0.977397,1.564511,0.100000,1.00,-1.637609,-0.868165,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,3.091313,-0.627749,0.309539,-0.344123,0.321434,-0.264207,0.207090,-1.494860,1.0,0.0,-0.947209,-0.086922,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.437130,1.172315,0.875191,-0.392816,-0.811837,-0.291810,-0.208737,0.150161,0.574307,-0.284132,0.363172,0.351546,0.136364,0.80,-0.984764,-0.309042,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,3.091313,1.191898,1.253929,-1.200952,1.406604,-1.358432,1.046996,2.268357,20.0,0.0,0.069322,-0.086922,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.768720,1.144183,0.294747,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.314651,-0.220448,-0.531720,-0.626094,1.680026,0.738234,1.686806,1.934743,-0.367261,1.143818,-1.009924,0.671991,0.033333,1.00,0.327633,0.032505,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,-3.716757,0.299307,-0.208059,-0.100354,-0.394045,-0.060357,0.207090,1.378593,1.0,1.0,-0.507837,0.343989,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.024409,-0.424676,26900.0,843300.0,0.91118

In [4]:
# Create a Neural Network class with flexible settings
class FlexibleRegressionNN(nn.Module):
    def __init__(self, input_size: int, hidden_layers_sizes: list, dropout_rate: float) -> None:
        super(FlexibleRegressionNN, self).__init__()
        layers = list()

        # Define first and n-1 hidden layers
        neuron_num = input_size
        for size in hidden_layers_sizes:
            layers.append(nn.Linear(neuron_num, size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            neuron_num = size

        layers.append(nn.Linear(neuron_num, 1))
        self.model = nn.Sequential(*layers)

    # Define forward loop for NN
    def forward(self, x):
        return self.model(x)

In [5]:
# Define function for calculation RMSE
def root_mean_squared_error(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [12]:
# Creating function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return test_loss

# Creating function for selection of best hyperparameters 
def objective(trial):
    # Defining hyperparameters
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 20)
    hidden_layer_sizes = [
        trial.suggest_int(f'layer_{i}_size', 32, 256)
        for i in range(num_hidden_layers)
    ]
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 5, 100)
    input_size = 59

    news_df = pd.read_csv('./prep_news_data.csv')

    X = news_df.iloc[:, :-1].values
    y = news_df.iloc[:, -1].values
    y = y.reshape(-1, 1)

    # Data normalization
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    # Splitting data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # Convertation data to PyTorch tensors
    train_features = torch.tensor(X_train, dtype=torch.float32)
    test_features = torch.tensor(X_test, dtype=torch.float32)
    train_targets = torch.tensor(y_train, dtype=torch.float32)
    test_targets = torch.tensor(y_test, dtype=torch.float32)    

    # Creating DataLoader
    train_data = TensorDataset(train_features, train_targets)
    test_data = TensorDataset(test_features, test_targets)

    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

    # Creating a model and defining of optimizer
    model = FlexibleRegressionNN(input_size, hidden_layer_sizes, dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Training launch and return the test loss
    return train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs)

# Settings of searching by Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Getting the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-03-13 20:28:14,080] A new study created in memory with name: no-name-cc16965a-188b-4a75-8ef0-82e5362e9e03
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_96862/2814934376.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_96862/2814934376.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/45, Train Loss: 0.0005, Test Loss: 0.0002
Epoch 2/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 3/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 4/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 5/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 6/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 7/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 8/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 9/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 10/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 11/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 12/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 13/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 14/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 15/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 16/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 17/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 18/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 19/45, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 20/45, Train Loss: 0.0002, Test Lo

[I 2025-03-13 20:28:37,044] Trial 0 finished with value: 0.0001666661719116658 and parameters: {'num_hidden_layers': 2, 'layer_0_size': 186, 'layer_1_size': 199, 'dropout_rate': 0.4259986028873003, 'learning_rate': 0.0008468771666289129, 'batch_size': 64, 'epochs': 45}. Best is trial 0 with value: 0.0001666661719116658.


Epoch 45/45, Train Loss: 0.0002, Test Loss: 0.0002


/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_96862/2814934376.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_96862/2814934376.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/12, Train Loss: 0.0003, Test Loss: 0.0002
Epoch 2/12, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 3/12, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 4/12, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 5/12, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 6/12, Train Loss: 0.0002, Test Loss: 0.0002
Epoch 7/12, Train Loss: 0.0002, Test Loss: 0.0002


[W 2025-03-13 20:28:49,183] Trial 1 failed with parameters: {'num_hidden_layers': 17, 'layer_0_size': 86, 'layer_1_size': 229, 'layer_2_size': 176, 'layer_3_size': 256, 'layer_4_size': 65, 'layer_5_size': 213, 'layer_6_size': 64, 'layer_7_size': 52, 'layer_8_size': 237, 'layer_9_size': 236, 'layer_10_size': 152, 'layer_11_size': 172, 'layer_12_size': 197, 'layer_13_size': 240, 'layer_14_size': 77, 'layer_15_size': 158, 'layer_16_size': 143, 'dropout_rate': 0.22521904371576032, 'learning_rate': 0.00015444564504123095, 'batch_size': 256, 'epochs': 12} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_96862/2814934376.py", line 77, in objective
    return train_and_evaluate_model(m

KeyboardInterrupt: 